# Extracting Environmental Data

Environmental data can be downloaded from [WorldClim](https://www.worldclim.org/data/worldclim21.html) directly or extracted using the following code:

## Bioclimatic Variable Data for SSP Scenario

In [1]:
import xarray as xr
import rioxarray as rio
import numpy as np
import os, requests, zipfile, io
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Setting the boundaries of the study region
min_lon = 140
min_lat = -44
max_lon = 154
max_lat = -28

# Projected times
times = ['2021-2040','2041-2060','2061-2080','2081-2100']

# SSP scenarios
ssp = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

# Resolution
res = ['2.5m', '30s']

In [3]:
def getDataset(ssp, time, res):
    
    '''
    This function retrieves the dataset for the specified SSP scenario, projected time period and resolution.
    
    args:
    - ssp: the SSP(Shared Socioeconomic Pathway) scenario, e.g. ssp126
    - time: the projected climatic time period, e.g. 2021-2040
    - res: the arc-minute resultion of the data, e.g. 2.5m    
    '''
    
    # WorldClim projections for GCM ACCESS-CM2
    url='https://geodata.ucdavis.edu/cmip6/'+res+'/ACCESS-CM2/'+ssp+'/wc2.1_'+res+'_bioc_ACCESS-CM2_'+ssp+'_'+time+'.tif'
    ds = rio.open_rasterio(url,masked=True,band_as_variable=True).squeeze()
    ds = ds.to_dataset('band')
    ds2=ds.rename(name_dict={1:'bio1', 2:'bio2', 3:'bio3', 4:'bio4', 5:'bio5', 6:'bio6', 7:'bio7',
                                      8:'bio8', 9:'bio9', 10:'bio10', 11:'bio11', 12:'bio12', 13:'bio13', 14:'bio14',
                                      15:'bio15', 16:'bio16', 17:'bio17', 18:'bio18', 19:'bio19'})
    
    # This study only examines 7 of the 19 variables (bio1, bio4, bio5, bio6, bio12, bio13, bio14) so we drop all others
    ds2 = ds2.drop(['bio2', 'bio3', 'bio7', 'bio8', 'bio9', 'bio10', 'bio11', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19'])

    # The dataset is clipped to the region of study
    ds2_clipped = ds2.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)
    ds2_clipped

    return ds2_clipped

Calling the function

In [4]:
SSP245_41_60 = getDataset(ssp[1], times[1], res[0])
SSP245_81_00 = getDataset(ssp[1], times[3], res[0])
SSP585_41_60 = getDataset(ssp[3], times[1], res[0])
SSP585_81_00 = getDataset(ssp[3], times[3], res[0])

Saving the datasets as netcdf files

In [44]:
# Create directories to save the netcdf files in their corresponding folders
os.mkdir('data')
os.mkdir('data/SSP')
os.mkdir('data/SSP/SSP245_41-60/')
os.mkdir('data/SSP/SSP245_81-00/')
os.mkdir('data/SSP/SSP585_41-60/')
os.mkdir('data/SSP/SSP585_81-00/')

SSP245_41_60.to_netcdf('data/SSP/SSP245_41-60/SSP245_41_60.nc')
SSP245_81_00.to_netcdf('data/SSP/SSP245_81-00/SSP245_81_00.nc')
SSP585_41_60.to_netcdf('data/SSP/SSP585_41-60/SSP585_41_60.nc')
SSP585_81_00.to_netcdf('data/SSP/SSP585_81-00/SSP585_81_00.nc')

## Elevation Data

In [5]:
# Setting the boundaries of the study region
min_lon = 140
min_lat = -44
max_lon = 154
max_lat = -28

# Setting the desired resolution
res = '2.5m'

# WorldClim elevation data derived from STRM elevation data
url="https://geodata.ucdavis.edu/climate/worldclim/2_1/base/wc2.1_"+res+"_elev.zip"

# Getting the zip file
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Create a new folder to extract the contents into
newpath = '/elev' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
z.extractall(newpath)

Then we open the TIF File and clip it to the region of interest:

In [6]:
ds = rio.open_rasterio('/elev/wc2.1_'+res+'_elev.tif',masked=True,band_as_variable=True).squeeze()
ds
ds2_clipped = ds.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)
ds2_clipped

<xarray.DataArray (y: 384, x: 336)>
[129024 values with dtype=float32]
Coordinates:
    band         int32 1
  * x            (x) float64 140.0 140.1 140.1 140.1 ... 153.9 153.9 153.9 154.0
  * y            (y) float64 -28.02 -28.06 -28.1 -28.15 ... -43.9 -43.94 -43.98
    spatial_ref  int32 0
Attributes:
    STATISTICS_MAXIMUM:  7412
    STATISTICS_MEAN:     1.#SNAN
    STATISTICS_MINIMUM:  -415
    STATISTICS_STDDEV:   1.#SNAN
    scale_factor:        1.0
    add_offset:          0.0

Converting the dataarray into a dataset and saving it as a netCDF

In [30]:
dsElev = xr.Dataset({
    'elev': xr.DataArray(
        data   = ds2_clipped.values,   # enter elev data here
        dims   = ['y','x'],
        coords = {'x': ds2_clipped.x, 'y': ds2_clipped.y},
    ),
},
)
dsElev.to_netcdf('data/elev.nc')